In [1]:
import tensorflow as tf 

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [11]:
import numpy as np
import tensorflow as tf
from scipy.stats import entropy

def calculate_inception_score(images, classifier, batch_size=32, splits=10):
    """
    Computes the Inception Score (IS) for generated images using a given classifier.

    Parameters:
        images (numpy.ndarray): 
            Generated images with shape (N, H, W, C). 
            For example, MNIST, FASION: (N, 28, 28, 1), CIFAR: (N, 32, 32, 3).
        classifier (tf.keras.Model): 
            A pretrained classifier (e.g., Inception-V3 for CIFAR or a CNN for MNIST/Fashion-MNIST).
        batch_size (int): 
            Batch size for model predictions.
        splits (int): 
            Number of splits for IS computation.

    Returns:
        tuple: (mean_inception_score, std_inception_score)
    """
    # Determine the expected number of channels and spatial dimensions from the classifier's input shape.
    # classifier.input_shape is generally like (None, H_expected, W_expected, C_expected)
    expected_height, expected_width, expected_channels = classifier.input_shape[1:4]

    # Adjust the channel dimension if needed:
    if images.shape[-1] != expected_channels:
        if images.shape[-1] == 1 and expected_channels == 3:
            images = np.repeat(images, 3, axis=-1)
        elif images.shape[-1] == 3 and expected_channels == 1:
            images = np.mean(images, axis=-1, keepdims=True)
        else:
            raise ValueError(
                f"Incompatible channel dimensions: images have {images.shape[-1]} channels, but classifier expects {expected_channels}."
            )

    # Normalize images to [0, 1] if they are not already
    if images.max() > 1:
        images = images.astype(np.float32) / 255.0

    # Resize images if their spatial dimensions don't match the classifier's expected dimensions.
    if images.shape[1] != expected_height or images.shape[2] != expected_width:
        # Convert images to a TensorFlow tensor, resize, then convert back to numpy.
        images = tf.image.resize(images, (expected_height, expected_width)).numpy()

    num_images = images.shape[0]
    preds = []

    # Process images in batches.
    for i in range(0, num_images, batch_size):
        batch = images[i : i + batch_size]
        # Get classifier predictions (logits)
        logits = classifier.predict(batch, verbose=0)
        # Convert logits to probabilities via softmax
        prob = tf.nn.softmax(logits).numpy()
        preds.append(prob)
        
    preds = np.concatenate(preds, axis=0)  # Shape: (N, num_classes)

    # Compute the Inception Score using KL divergence
    split_scores = []
    split_size = num_images // splits
    for i in range(splits):
        part = preds[i * split_size : (i + 1) * split_size]
        p_y = np.mean(part, axis=0)  # Marginal distribution over classes
        kl_divs = [entropy(p, p_y) for p in part]
        split_score = np.exp(np.mean(kl_divs))
        split_scores.append(split_score)
    
    return np.mean(split_scores), np.std(split_scores)


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

def train_mnist_classifier(epochs=5, batch_size=128):
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # Expand dimensions to include the channel (MNIST images are 28x28, grayscale)
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    
    # Normalize the images to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    # Convert labels to one-hot vectors
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
    
    # Build a simple CNN model
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        # Final Dense layer without activation since we use logits
        layers.Dense(10)
    ])
    
    # Compile the model with the loss function expecting logits (from_logits=True)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(x_train, y_train, 
              epochs=epochs, 
              batch_size=batch_size, 
              validation_data=(x_test, y_test))
    
    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")
    
    return model

# Train the MNIST classifier
# mnist_classifier = train_mnist_classifier(epochs=5)

# Optionally, save the model if you'd like to reuse it later:
# mnist_classifier.save("mnist_cnn_model.h5")
# mnist_classifier = tf.keras.models.load_model("mnist_cnn_model.h5")  # Example path



In [13]:
# Load a trained CNN classifier for MNIST
generated_mnist_images = np.random.rand(1000, 28, 28, 1)  # Fake grayscale images

mean_is, std_is = calculate_inception_score(generated_mnist_images, mnist_classifier)
print(f"Inception Score: {mean_is:.4f} ± {std_is:.4f}")


Inception Score: 1.1078 ± 0.0072


In [18]:
# Load a pre-trained InceptionV3 model (trained on ImageNet)
inception_model = tf.keras.applications.InceptionV3(weights="imagenet", include_top=True)
generated_cifar_images = np.random.rand(20, 32, 32, 3)  # Fake CIFAR images

mean_is, std_is = calculate_inception_score(generated_cifar_images, inception_model)
print(f"Inception Score: {mean_is:.4f} ± {std_is:.4f}")


Inception Score: 1.0000 ± 0.0000


In [1]:
import os 
os.listdir('./../code/experiments/')

['.ipynb_checkpoints',
 'analyze_created_images.ipynb',
 'analyze_created_labels.ipynb',
 'check_datasets.ipynb',
 'ConditionalGAN.ipynb',
 'models',
 'n-mad-gan.ipynb',
 'test_FID_score.ipynb',
 'test_inception_score.ipynb',
 'vanilla_gan_test']